In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, AdamW
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from tqdm import tqdm
import pandas as pd
import numpy as np
import importlib

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

torch.backends.cudnn.benchmark = True
# torch.cuda.set_per_process_memory_fraction(0.7)
torch.cuda.empty_cache()

Using device: cuda


In [2]:
DSET_CLASS = torchvision.datasets.CIFAR10
NUM_CLASSES = 10

transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

testset = DSET_CLASS(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=512, shuffle=False, num_workers=2)

# define dataset for attack model that shadow models will generate
print("mapped classes to ids:", testset.class_to_idx)


Files already downloaded and verified
mapped classes to ids: {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [3]:
# Training multiple shadow models
model_architecture = importlib.import_module("torchvision.models")
model_class = getattr(model_architecture, "resnet18")
criterion = nn.CrossEntropyLoss()

# Train Target Model
target_model = model_class(num_classes=NUM_CLASSES)
target_model.fc = nn.Linear(in_features=target_model.fc.in_features, out_features=NUM_CLASSES, bias=True)
target_model = target_model.to(device)
optimizer = AdamW(target_model.parameters(), lr=0.001, weight_decay=0.00001)

target_train_indices = np.random.choice(len(testset), 5000, replace=False)
target_eval_indices = np.setdiff1d(np.arange(len(testset)), target_train_indices)


In [4]:
os.makedirs("./models", exist_ok=True)
pd.DataFrame(target_train_indices, columns=["index"]).to_csv(
    "./models/target_train_indices.csv", index=False
)

subset_tgt_train = Subset(testset, target_train_indices)
subset_tgt_eval = Subset(testset, target_eval_indices)

subset_tgt_train_loader = DataLoader(subset_tgt_train, batch_size=256, shuffle=True, num_workers=2)
subset_tgt_eval_loader = DataLoader(subset_tgt_eval, batch_size=512, shuffle=False, num_workers=2)

run_name = f"target_model_resnet18_cifar10"

In [5]:
class EarlyStopPatience(nn.Module):
    def __init__(self, patience=10):
        super(EarlyStopPatience, self).__init__()
        self.patience = patience
        self.counter = 0
        self.best_loss = None

    def __call__(self, loss):
        if self.best_loss is None:
            self.best_loss = loss
        elif loss > self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_loss = loss
            self.counter = 0
        return False

In [ ]:
EPOCHS = 100
SAVE_PATH = "./models"


early_stop_acc1 = EarlyStopPatience(patience=10)
best_valid_acc = 0
best_valid_loss = 10


if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

for epoch in range(EPOCHS):
    target_model.train()
    train_loss = 0
    train_acc = 0
    for i, (inputs, labels) in enumerate(subset_tgt_train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = target_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()

    train_loss /= len(subset_tgt_train_loader)
    train_acc /= len(subset_tgt_train_loader.dataset)

    target_model.eval()
    valid_loss = 0
    valid_acc = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(subset_tgt_eval_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = target_model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            valid_acc += (outputs.argmax(1) == labels).sum().item()

    valid_loss /= len(subset_tgt_eval_loader)
    valid_acc /= len(subset_tgt_eval_loader.dataset)

    print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        torch.save(target_model.state_dict(), f"{SAVE_PATH}/{run_name}.pth")
        print(f"Saved model with valid acc: {valid_acc:.4f} -> {SAVE_PATH}/{run_name}.pth")

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss

    if early_stop_acc1(valid_acc):
        print("Early stopping")
        break

print("Loading best model...")
print(f"Best valid acc: {best_valid_acc:.4f}")
print(f"Best valid loss: {best_valid_loss:.4f}")

target_model.load_state_dict(torch.load(f"{SAVE_PATH}/{run_name}.pth"))



Epoch 1/100, Train Loss: 0.6424, Train Acc: 0.7858, Valid Loss: 1.5764, Valid Acc: 0.4788
Saved model with valid acc: 0.4788 -> ./models/shadow_model_0_resnet18_cifar10.pth
Epoch 2/100, Train Loss: 0.6419, Train Acc: 0.7892, Valid Loss: 1.5723, Valid Acc: 0.4846
Saved model with valid acc: 0.4846 -> ./models/shadow_model_0_resnet18_cifar10.pth
Epoch 3/100, Train Loss: 0.6443, Train Acc: 0.7838, Valid Loss: 1.5747, Valid Acc: 0.4836
Epoch 4/100, Train Loss: 0.6432, Train Acc: 0.7860, Valid Loss: 1.5755, Valid Acc: 0.4838
Epoch 5/100, Train Loss: 0.6407, Train Acc: 0.7852, Valid Loss: 1.5733, Valid Acc: 0.4834
Epoch 6/100, Train Loss: 0.6410, Train Acc: 0.7866, Valid Loss: 1.5768, Valid Acc: 0.4832
Epoch 7/100, Train Loss: 0.6465, Train Acc: 0.7848, Valid Loss: 1.5747, Valid Acc: 0.4818
Epoch 8/100, Train Loss: 0.6472, Train Acc: 0.7852, Valid Loss: 1.5722, Valid Acc: 0.4834
Epoch 9/100, Train Loss: 0.6439, Train Acc: 0.7860, Valid Loss: 1.5749, Valid Acc: 0.4836
Epoch 10/100, Train Loss

/tmp/ipykernel_108496/3042342292.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  target_model.load_state_dict(torch.load(f"{SAVE_PATH}/{run_name}.pth"))


<All keys matched successfully>

In [ ]:
shadow_set = DSET_CLASS(root='./data', train=True, download=True, transform=transform)
shadow_loader = DataLoader(shadow_set, batch_size=256, shuffle=True, num_workers=2)

# define dataset for attack model that shadow models will generate
columns_attack_sdet = [f"top_{index}_prob" for index in range(10)]
df_attack_dset = pd.DataFrame({}, columns=columns_attack_sdet + ["is_member"])

# random subset for shadow model train & validation from the CIFAR shadow_set
list_train_loader = []
list_eval_loader = []
list_test_loader = []

NUM_SHODOW_MODELS = 64

for _ in range(NUM_SHODOW_MODELS):
    train_indices = np.random.choice(len(shadow_set), 2500, replace=False)
    eval_indices = np.setdiff1d(np.arange(len(shadow_set)), train_indices)
    eval_indices = np.random.choice(eval_indices, 2500, replace=False)
    test_indices = np.setdiff1d(
        np.arange(len(shadow_set)), np.concatenate((train_indices, eval_indices))
    )
    test_indices = np.random.choice(test_indices, 2500, replace=False)

    subset_train = Subset(shadow_set, train_indices)
    subset_eval = Subset(shadow_set, eval_indices)
    subset_test = Subset(shadow_set, test_indices)

    subset_train_loader = DataLoader(subset_train, batch_size=256, shuffle=True, num_workers=2)
    subset_eval_loader = DataLoader(subset_eval, batch_size=256, shuffle=False, num_workers=2)
    subset_test_loader = DataLoader(subset_test, batch_size=256, shuffle=False, num_workers=2)

    list_train_loader.append(subset_train_loader)
    list_eval_loader.append(subset_eval_loader)
    list_test_loader.append(subset_test_loader)

model_architecture = importlib.import_module("torchvision.models")
model_class = getattr(model_architecture, "resnet18")
criterion = nn.CrossEntropyLoss()

for shadow_number, shadow_loader in enumerate(tqdm(list_train_loader)):
    print(f"Training shadow model {shadow_number}/{NUM_SHODOW_MODELS}")
    evalloader = list_eval_loader[shadow_number]
    testloader = list_test_loader[shadow_number]

    shadow_model = model_class(pretrained=False)
    shadow_model.fc = nn.Linear(in_features=shadow_model.fc.in_features, out_features=NUM_CLASSES, bias=True)
    shadow_model = shadow_model.to(device)
    optimizer = AdamW(shadow_model.parameters(), lr=0.001, weight_decay=0.00001)

    run_name = f"shadow_model_{shadow_number}_resnet18_cifar10_loss_{best_valid_loss:.4f}_acc_{best_valid_acc:.4f}"

    early_stop_acc1 = EarlyStopPatience(patience=10)
    best_valid_acc = 0
    best_valid_loss = 10

    for epoch in range(EPOCHS):
        shadow_model.train()
        train_loss = 0
        train_acc = 0
        for i, (inputs, labels) in enumerate(shadow_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = shadow_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += (outputs.argmax(1) == labels).sum().item()

        train_loss /= len(shadow_loader)
        train_acc /= len(shadow_loader.dataset)

        shadow_model.eval()
        valid_loss = 0
        valid_acc = 0
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(evalloader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = shadow_model(inputs)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                valid_acc += (outputs.argmax(1) == labels).sum().item()

        valid_loss /= len(evalloader)
        valid_acc /= len(evalloader.dataset)

        print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}")

        if valid_acc > best_valid_acc:
            best_valid_acc = valid_acc
            torch.save(shadow_model.state_dict(), f"{SAVE_PATH}/{run_name}.pth")
            print(f"Saved model with valid acc: {valid_acc:.4f} -> {SAVE_PATH}/{run_name}.pth")

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss

        if early_stop_acc1(valid_acc):
            print("Early stopping")
            break
    

Files already downloaded and verified


  0%|          | 0/64 [00:00<?, ?it/s]/home/nurda/repos/fyp/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nurda/repos/fyp/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training shadow model 0/64
Epoch 1/100, Train Loss: 2.1316, Train Acc: 0.2624, Valid Loss: 2.2360, Valid Acc: 0.1644
Saved model with valid acc: 0.1644 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.2100.pth
Epoch 2/100, Train Loss: 1.4706, Train Acc: 0.4704, Valid Loss: 2.4899, Valid Acc: 0.1764
Saved model with valid acc: 0.1764 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.2100.pth
Epoch 3/100, Train Loss: 0.9563, Train Acc: 0.6680, Valid Loss: 2.6195, Valid Acc: 0.2428
Saved model with valid acc: 0.2428 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.2100.pth
Epoch 4/100, Train Loss: 0.5151, Train Acc: 0.8332, Valid Loss: 2.5874, Valid Acc: 0.3072
Saved model with valid acc: 0.3072 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.2100.pth
Epoch 5/100, Train Loss: 0.2232, Train Acc: 0.9344, Valid Loss: 2.4533, Valid Acc: 0.3928
Saved model with valid acc: 0.3928 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.21

  2%|▏         | 1/64 [00:25<27:10, 25.88s/it]

Saved model with valid acc: 0.4248 -> ./models/shadow_model_0_resnet18_cifar10_loss_2.2789_acc_0.2100.pth
Early stopping
Training shadow model 1/64
Epoch 1/100, Train Loss: 2.1141, Train Acc: 0.2644, Valid Loss: 2.4008, Valid Acc: 0.1092
Saved model with valid acc: 0.1092 -> ./models/shadow_model_1_resnet18_cifar10_loss_2.2360_acc_0.4248.pth
Epoch 2/100, Train Loss: 1.4704, Train Acc: 0.4888, Valid Loss: 2.7168, Valid Acc: 0.1292
Saved model with valid acc: 0.1292 -> ./models/shadow_model_1_resnet18_cifar10_loss_2.2360_acc_0.4248.pth
Epoch 3/100, Train Loss: 0.8988, Train Acc: 0.6896, Valid Loss: 2.8582, Valid Acc: 0.2020
Saved model with valid acc: 0.2020 -> ./models/shadow_model_1_resnet18_cifar10_loss_2.2360_acc_0.4248.pth
Epoch 4/100, Train Loss: 0.4477, Train Acc: 0.8544, Valid Loss: 2.4643, Valid Acc: 0.2996
Saved model with valid acc: 0.2996 -> ./models/shadow_model_1_resnet18_cifar10_loss_2.2360_acc_0.4248.pth
Epoch 5/100, Train Loss: 0.2032, Train Acc: 0.9436, Valid Loss: 2.69

...

Saved model with valid acc: 0.4024 -> ./models/shadow_model_23_resnet18_cifar10_loss_2.2855_acc_0.4132.pth
Early stopping
Training shadow model 24/64
Epoch 1/100, Train Loss: 2.0861, Train Acc: 0.2708, Valid Loss: 2.2751, Valid Acc: 0.1408
Saved model with valid acc: 0.1408 -> ./models/shadow_model_24_resnet18_cifar10_loss_2.4518_acc_0.4024.pth
Epoch 2/100, Train Loss: 1.4400, Train Acc: 0.4880, Valid Loss: 2.5832, Valid Acc: 0.1812
Saved model with valid acc: 0.1812 -> ./models/shadow_model_24_resnet18_cifar10_loss_2.4518_acc_0.4024.pth
Epoch 3/100, Train Loss: 0.9119, Train Acc: 0.6844, Valid Loss: 2.3710, Valid Acc: 0.2540
Saved model with valid acc: 0.2540 -> ./models/shadow_model_24_resnet18_cifar10_loss_2.4518_acc_0.4024.pth
Epoch 4/100, Train Loss: 0.4853, Train Acc: 0.8468, Valid Loss: 2.4930, Valid Acc: 0.2936
Saved model with valid acc: 0.2936 -> ./models/shadow_model_24_resnet18_cifar10_loss_2.4518_acc_0.4024.pth
Epoch 5/100, Train Loss: 0.2431, Train Acc: 0.9324, Valid Loss

 39%|███▉      | 25/64 [11:17<17:56, 27.60s/it]

Epoch 11/100, Train Loss: 0.1126, Train Acc: 0.9640, Valid Loss: 2.8688, Valid Acc: 0.4128
Early stopping
Training shadow model 25/64
Epoch 1/100, Train Loss: 2.1251, Train Acc: 0.2464, Valid Loss: 2.2618, Valid Acc: 0.1644
Saved model with valid acc: 0.1644 -> ./models/shadow_model_25_resnet18_cifar10_loss_2.2751_acc_0.4160.pth
Epoch 2/100, Train Loss: 1.4456, Train Acc: 0.4932, Valid Loss: 2.6443, Valid Acc: 0.1804
Saved model with valid acc: 0.1804 -> ./models/shadow_model_25_resnet18_cifar10_loss_2.2751_acc_0.4160.pth
Epoch 3/100, Train Loss: 0.8980, Train Acc: 0.6880, Valid Loss: 2.2297, Valid Acc: 0.2540
Saved model with valid acc: 0.2540 -> ./models/shadow_model_25_resnet18_cifar10_loss_2.2751_acc_0.4160.pth
Epoch 4/100, Train Loss: 0.4736, Train Acc: 0.8528, Valid Loss: 2.2462, Valid Acc: 0.3444
Saved model with valid acc: 0.3444 -> ./models/shadow_model_25_resnet18_cifar10_loss_2.2751_acc_0.4160.pth
Epoch 5/100, Train Loss: 0.2314, Train Acc: 0.9320, Valid Loss: 2.5196, Valid 

 41%|████      | 26/64 [11:41<16:49, 26.56s/it]

Epoch 11/100, Train Loss: 0.2127, Train Acc: 0.9260, Valid Loss: 3.1753, Valid Acc: 0.3556
Early stopping
Training shadow model 26/64
Epoch 1/100, Train Loss: 2.0970, Train Acc: 0.2724, Valid Loss: 2.4211, Valid Acc: 0.1228
Saved model with valid acc: 0.1228 -> ./models/shadow_model_26_resnet18_cifar10_loss_2.2297_acc_0.4052.pth
Epoch 2/100, Train Loss: 1.3959, Train Acc: 0.5084, Valid Loss: 2.6420, Valid Acc: 0.1692
Saved model with valid acc: 0.1692 -> ./models/shadow_model_26_resnet18_cifar10_loss_2.2297_acc_0.4052.pth
Epoch 3/100, Train Loss: 0.8978, Train Acc: 0.6880, Valid Loss: 2.4668, Valid Acc: 0.2572
Saved model with valid acc: 0.2572 -> ./models/shadow_model_26_resnet18_cifar10_loss_2.2297_acc_0.4052.pth


In [ ]:
def make_member_nonmember(finetuned_model, trainloader, testloader, criterion, device):
    """ 
    - finetuned_model: finetuned shadow model
    - trainloader: member
    - testloader: non-member
    - criterion: loss function
    - device: cuda or cpu
    """

    member_dset = []
    non_member_dset = []

    finetuned_model.eval()
    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(trainloader)):
            images = images.to(device)
            labels = labels.to(device)

            # compute output
            output = finetuned_model(images)
            prob = F.softmax(output, dim=1)  # softmax logits

            # get top inferred classes probability and append to member_dset
            top_p, top_class = prob.topk(CFG.topk_num_accessible_probs, dim=1)
            top_p = top_p.cpu().detach().numpy()  # detach from cuda
            # loss = criterion(output, labels)
            member_dset.append(top_p)

        for i, (images, labels) in enumerate(tqdm(testloader)):
            images = images.to(device)
            labels = labels.to(device)

            # compute output
            output = finetuned_model(images)
            prob = F.softmax(output, dim=1)  # softmax logits

            # get top inferred classes probability and append to member_dset
            top_p, top_class = prob.topk(CFG.topk_num_accessible_probs, dim=1)
            top_p = top_p.cpu().detach().numpy()  # detach from cuda

            # append to non_member_dset
            non_member_dset.append(top_p)

    max_len = max(max(len(x) for x in member_dset), max(len(x) for x in non_member_dset))
    member_dset = [np.pad(x, ((0, max_len - len(x)), (0, 0)), mode='constant') for x in member_dset]
    non_member_dset = [np.pad(x, ((0, max_len - len(x)), (0, 0)), mode='constant') for x in non_member_dset]

    # change into numpy array type
    member_dset, non_member_dset = np.array(member_dset), np.array(non_member_dset)

    # return as dataset row x number of accessible probabilities: (ex) 25000 x 5, 25000 x 5
    return np.concatenate(member_dset, axis=0), np.concatenate(non_member_dset, axis=0)



In [ ]:
member_dset, non_member_dset = make_member_nonmember(
        finetuned_model, shadow_loader, testloader, criterion, device
    )
df_member = pd.DataFrame(member_dset, columns=columns_attack_sdet)
df_member["is_member"] = 1
df_non_member = pd.DataFrame(non_member_dset, columns=columns_attack_sdet)
df_non_member["is_member"] = 0

df_attack_dset = pd.concat([df_member, df_non_member], axis=0)
ATTACK_SAVE_PATH = "./attack"
df_attack_dset.to_csv(
    f"{ATTACK_SAVE_PATH}/{shadow_model.__class__.__name__}_pretrained_False_num_shadow_{NUM_SHODOW_MODELS}_CIFAR{NUM_CLASSES}_attack_dset.csv",
    index=False,
)

shadow_model.cpu()
del shadow_model, optimizer, shadow_loader, evalloader, testloader
torch.cuda.empty_cache()